## Week 3 - Image Recognition

### Heather Tweedie, 1/2/23

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import math

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

import matplotlib.style #Some style nonsense
import matplotlib as mpl #Some more style nonsense

#Set default figure size
#mpl.rcParams['figure.figsize'] = [12.0, 8.0] #Inches... of course it is inches
mpl.rcParams["legend.frameon"] = False
mpl.rcParams['figure.dpi']=200 # dots per inch


We first load the MNIST dataset from Keras, set the image scales correctly, and define the size of the images.

In [10]:
# load data
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# re-scale inputs
train_images=train_images/255.0
test_images=test_images/255.0

# check shape of datasets
print("Shape of training images:",train_images.shape)
print("Length of training set labels:",len(train_labels))
print("First label:",train_labels[0])
print("Shape of testing images:",test_images.shape)
print("Length of testing set labels:",len(test_labels))

image_x = len(train_images[0,:,0])
image_y = len(train_images[0,0,:])


Shape of training images: (60000, 28, 28)
Length of training set labels: 60000
First label: 5
Shape of testing images: (10000, 28, 28)
Length of testing set labels: 10000


We then define a number of functions for use in training and testing the model. `addNoise` adds a random noise contribution to each pixel of an image; `trainWithNoise` trains a model on noisy images; and `accuracyWithNoise` evaluates the accuracy of a model, testing it on noisy data.

In [11]:
def addNoise(image, y_noise):
    """
    Adds a random noise contribution drawn from a uniform distribution between 0 and a user-defined maximum to an image.
    
    Args:
        image: the image to which the noise will be added
        y_noise: the maximum value for the uniform distribution from which the random noise contribution will be drawn

    Returns:
        newImage: the new image with noise added
    """
    # get image dimensions
    image_x = 28
    image_y = 28

    newImage = np.empty([image_x, image_y])
    for i in range(image_x):
        for j in range(image_y):
            newImage[i,j] = image[i,j] + np.random.uniform(0, y_noise)

    return newImage

    

def trainWithNoise(model, image_x, image_y, y_noise, batchSize, nepochs):
    """
    Trains a model on MNIST image data with noise added, and evaluates it.
    
    Args:
        model: the model to be trained
        image_x: the x length of the image
        image_y: the y length of the image
        y_noise: the maximum value for the uniform distribution from which the random noise contribution will be drawn
        batchSize: the number of samples to be drawn during each step of the training
        nepochs: the number of epochs of training to be carried out
    
    Returns:
        test_loss: the cost function after training
        test_acc: the accuracy of the model based on testing on a test set of images
    """

    imagesWithNoise = np.empty([60000, image_x, image_y])
    for i in range(len(imagesWithNoise[:,0,0])):
        imagesWithNoise[i,:,:] = addNoise(train_images[i,:,:], y_noise)

    history = model.fit(imagesWithNoise, train_labels, batch_size = batchSize, epochs = nepochs)

    test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
    return test_loss, test_acc



def accuracyWithNoise(model, image_x, image_y, y_noise):
    """
    Evaluates a model trained on MNIST data on a test dataset with noise added.
    
    Args:
        model: the model to be evaluated
        image_x: the x length of the image
        image_y: the y length of the image
        y_noise: the maximum value for the uniform distribution from which the random noise contribution will be drawn
    
    Returns:
        test_loss: the cost of the model
        test_acc: the accuracy of the model
    """

    imagesWithNoise = np.empty([10000, image_x, image_y])
    for i in range(len(imagesWithNoise[:,0,0])):
        imagesWithNoise[i,:,:] = addNoise(test_images[i,:,:], y_noise)

    test_loss, test_acc = model.evaluate(imagesWithNoise, test_labels, verbose=2)
    return test_loss, test_acc

We now define and compile the network on which we will train the data:

In [12]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28,28)),
    keras.layers.Dense(128,activation='relu'),
    keras.layers.Dense(15)
])

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer=tf.keras.optimizers.SGD(learning_rate=1.0),
              metrics=['accuracy'])

Train the model on the MNIST dataset. Initially, using the parameters provided in the course material (layers: [128, 10]; batch size = 100, epochs = 30), the model achieved an accuracy of around 0.94, which is lower than the acceptable level of 0.95. I tried increasing the batch size to 120 and then 140, but this only achieved an inconsistent 0.945 - 0.95. By increasing the size of the final layer in the network from 10 to 15, the accuracy of the model grew substantially, achieving an accuracy of 0.9998 - 1 on the trianing dataset. In order to make the model training faster, I reduced the number of epochs from 30 to 10, as at this point the model was already achieving high accuracies, and reduced the batch size back down to 100. This also reduced the likelihood of overfitting.

In [ ]:
history = model.fit(train_images, train_labels, batch_size=100, epochs=10)

Evaluate accuracy of model for different noise levels:

In [ ]:
# test model accuracy against noisy data
noises = np.linspace(0, 2, 20)
accuracies = np.empty([len(noises)])

for i in range(len(noises)):
    print(f"y_noise = {noises[i]}:")
    test_loss, test_acc = accuracyWithNoise(model, image_x, image_y, noises[i])
    accuracies[i] = test_acc

fig, ax = plt.subplots()
ax.plot(noises, accuracies)
ax.set_xlabel('Maximum noise level')
ax.set_ylabel('Model accuracy')
ax.set_title('Model accuracy when evaluated against noisy data')

Define new model to be trained on noisy data. The final layer density is higher than previously to account for the increased difficulty the network will encounter during training.

In [13]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28,28)),
    keras.layers.Dense(128,activation='relu'),
    keras.layers.Dense(20)
])

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer=tf.keras.optimizers.SGD(learning_rate=1.0),
              metrics=['accuracy'])

Train model on noisy data. With increasing noise, the accuracy drops substantially. At 0.1, the accuracy with 0 noise is mostly maintained, however by 0.2 the accuracy drops to 0.1 be the end of the training. At earlier epochs however, the accuracy is ~0.7, but this drops rapidly with further training.

In [ ]:
# train model with noisy data
acc_1 = trainWithNoise(model, image_x, image_y, 0.1, 100, 20)
print(acc_1)

# test model accuracy against noisy data
noises = np.linspace(0, 2, 20)
accuracies = np.empty([len(noises)])

for i in range(len(noises)):
    print(f"y_noise = {noises[i]}:")
    test_loss, test_acc = accuracyWithNoise(model, image_x, image_y, noises[i])
    accuracies[i] = test_acc

fig, ax = plt.subplots()
ax.plot(noises, accuracies)
ax.set_xlabel('Maximum noise level')
ax.set_ylabel('Model accuracy')
ax.set_title('Model accuracy when evaluated against noisy data')


When tested against the noisy data, the model trained on noisy data performs much better than the model trained on clean data. Although when tested on clean data they perform similarly, at greater noise levels in the test data, the accuracy of the model trained on noisy data drops much slower than that of the model trained on clean data. 

This is extra! Remove before submitting if incomplete

In [14]:
training_noises = np.linspace(0, 0.5, 6)
num_noises = 10
all_accuracies = np.empty([len(training_noises), num_noises])

for j in range(len(training_noises)):

    # train model with noisy data
    acc, loss = trainWithNoise(model, image_x, image_y, training_noises[j], 100, 20)

    # test model accuracy against noisy data
    noises = np.linspace(0, 2, num_noises)
    for i in range(len(noises)):
        print(f"y_noise = {noises[i]}:")
        test_loss, test_acc = accuracyWithNoise(model, image_x, image_y, noises[i])
        all_accuracies[j,i] = test_acc


#fig, ax = plt.subplots()
#ax.plot(noises, accuracies_1)
#ax.set_xlabel('Maximum noise level')
#ax.set_ylabel('Model accuracy')
#ax.set_title('Model accuracy when evaluated against noisy data')

Epoch 1/20
600/600 [==============================] - 4s 5ms/step - loss: 0.4917 - accuracy: 0.8564
Epoch 2/20
600/600 [==============================] - 1s 2ms/step - loss: 0.1728 - accuracy: 0.9491
Epoch 3/20
600/600 [==============================] - 1s 2ms/step - loss: 0.1344 - accuracy: 0.9595
Epoch 4/20
600/600 [==============================] - 1s 2ms/step - loss: 0.1150 - accuracy: 0.9652
Epoch 5/20
600/600 [==============================] - 1s 2ms/step - loss: 0.1033 - accuracy: 0.9684
Epoch 6/20
600/600 [==============================] - 1s 2ms/step - loss: 0.0918 - accuracy: 0.9722
Epoch 7/20
600/600 [==============================] - 1s 2ms/step - loss: 0.0843 - accuracy: 0.9742
Epoch 8/20
600/600 [==============================] - 1s 2ms/step - loss: 0.0784 - accuracy: 0.9761
Epoch 9/20
600/600 [==============================] - 1s 2ms/step - loss: 0.0742 - accuracy: 0.9772
Epoch 10/20
600/600 [==============================] - 3s 5ms/step - loss: 0.0681 - accuracy: 0.9790

KeyboardInterrupt: 